### 14.1.1 Reader 对象


In [1]:
import csv #csv 模块是 Python 自带的，所以不需要安装就可以导入它
exampleFile = open('example.csv')  #要用 csv 模块读取 CSV 文件，首先用 open()函数打开它
exampleReader = csv.reader(exampleFile) 
#就像打开任何其他文本文件一样。但是，不用在 open()返回的 File 对象上调用 read()或 readlines()方法，而是将它传递给 csv.reader()函数
exampleData = list(exampleReader)
#要访问 Reader 对象中的值，最直接的方法，就是将它转换成一个普通 Python 列表， 即将它传递给 list()
exampleData

[['4/5/2014 13:34', 'Apples', '73'],
 ['4/5/2014 3:41', 'Cherries', '85'],
 ['4/6/2014 12:46', 'Pears', '14'],
 ['4/8/2014 8:59', 'Oranges', '52'],
 ['4/10/2014 2:07', 'Apples', '152'],
 ['4/10/2014 18:10', 'Bananas', '23'],
 ['4/10/2014 2:40', 'Strawberries', '98']]

In [2]:
#既然已经将 CSV 文件表示为列表的列表，就可以用表达式 exampleData[row][col]来访问特定行和列的值。
exampleData[0][1]

'Apples'

### 14.1.2 在 for 循环中，从 Reader 对象读取数据


In [3]:
#对于大型的 CSV 文件，你需要在一个 for 循环中使用 Reader 对象。这样避免将整个文件一次性装入内存。
exampleFile = open('example.csv')
exampleReader = csv.reader(exampleFile)
for row in exampleReader:
    print('Row #' + str(exampleReader.line_num) + ' ' + str(row))

Row #1 ['4/5/2014 13:34', 'Apples', '73']
Row #2 ['4/5/2014 3:41', 'Cherries', '85']
Row #3 ['4/6/2014 12:46', 'Pears', '14']
Row #4 ['4/8/2014 8:59', 'Oranges', '52']
Row #5 ['4/10/2014 2:07', 'Apples', '152']
Row #6 ['4/10/2014 18:10', 'Bananas', '23']
Row #7 ['4/10/2014 2:40', 'Strawberries', '98']


### 14.1.3 Writer 对象


In [4]:
#Writer 对象让你将数据写入 CSV 文件。要创建一个 Writer 对象，就使用csv.writer()函数

In [5]:
outputFile = open('output.csv', 'w', newline='')
outputWriter = csv.writer(outputFile)
outputWriter.writerow(['spam', 'eggs', 'bacon', 'ham'])
#首先，调用 open()并传入'w'，以写模式打开一个文件。这将创建对象。
#然后将它传递给 csv.writer()，创建一个 Writer 对象。
#在 Windows 上，需要为 open()函数的 newline 关键字参数传入一个空字符串。
#这样做的技术原因超出了本书的范围。如果忘记设置 newline 关键字参数， output.csv中的行距将有两倍

21

In [6]:
outputWriter.writerow(['Hello, world!', 'eggs', 'bacon', 'ham'])

32

In [7]:
outputWriter.writerow(['Hello, world!', 'eggs', 'bacon', 'ham'])

32

In [8]:
outputWriter.writerow([1, 2, 3.141592, 4])

16

In [9]:
outputFile.close()

### delimiter 和 lineterminator 关键字参数

In [10]:
#用制表符代替逗号来分隔单元格，并希望有两倍行距
csvFile = open('example.tsv', 'w', newline='')
csvWriter = csv.writer(csvFile, delimiter='\t', lineterminator='\n\n')
csvWriter.writerow(['apples', 'oranges', 'grapes'])

23

In [11]:
csvWriter.writerow(['eggs', 'bacon', 'ham'])
csvWriter.writerow(['spam', 'spam', 'spam', 'spam', 'spam', 'spam'])

31

In [12]:
csvFile.close()

###### 
行终止字符是出现在行末的字符。默认情况下，行终止字符是换行符。   
你可以利用 csv.writer()的 delimiter 和 lineterminator关键字参数，将这些字符改成不同的值。  
传入 delimeter='\t'和 lineterminator='\n\n'，这将单元格之间的字符改变为制表符，  
将行之间的字符改变为两个换行符。然后我们调用 writerow()三次，得到 3 行。  

## 14.2 项目：从 CSV 文件中删除表头

假设你有一个枯燥的任务，要删除几百CSV 文件的第一行。  
也许你会将它们送入一个自动化的过程，只需要数据，不需要每列顶部的表头。  
可以在Excel 中打开每个文件，删除第一行，并重新保存该文件，但这需要几个小时。让我们写一个程序来做这件事。  
该程序需要打开当前工作目录中所有扩展名为.csv 的文件，读取 CSV 文件的内容，  
并除掉第一行的内容重新写入同名的文件。这将用新的、无表头的内容替换CSV 文件的旧内容。  
警告 与往常一样，当你写程序修改文件时，一定要先备份这些文件，以防万一你的程序没有按期望的方式工作。你不希望意外地删除原始文件。  
总的来说，该程序必须做到以下几点：  
• 找出当前工作目录中的所有 CSV 文件。  
• 读取每个文件的全部内容。  
• 跳过第一行，将内容写入一个新的 CSV 文件。  
在代码层面上，这意味着该程序需要做到以下几点：  
• 循环遍历从 os.listdir()得到的文件列表，跳过非 CSV 文件。  
• 创建一个 CSV Reader 对象，读取该文件的内容，利用 line_num 属性确定要跳过哪一行。  
• 创建一个 CSV Writer 对象，将读入的数据写入新文件  

In [18]:
# 第 1 步：循环遍历每个 CSV 文件
import csv, os
os.makedirs('headerRemoved', exist_ok=True)
# Loop through every file in the current working directory.
for csvFilename in os.listdir('.'):
    if not csvFilename.endswith('.csv'):
        continue # skip non-csv files
    print('Removing header from ' + csvFilename + '...')
    
#os.makedirs()调用将创建 headerRemoved 文件夹，所有的无表头的 CSV 文件将写入该文件夹。
#针对 os.listdir('.')进行 for 循环完成了一部分任务，但这会遍历工作目录中的所有文件，
#所以需要在循环开始处添加一些代码，跳过扩展名不是.csv 的文件。
#如果遇到非 CSV 文件， continue 语句让循环转向下一个文件名。
#为了让程序运行时有一些输出，打印出一条消息说明程序在处理哪个 CSV 文件。

#第 2 步：读入 CSV 文件
    csvRows = []
    csvFileObj = open(csvFilename)
    readerObj = csv.reader(csvFileObj)
    for row in readerObj:
        if readerObj.line_num == 1:
            continue # skip first row
        csvRows.append(row)
    csvFileObj.close()

#Reader 对象的 line_num 属性可以用来确定当前读入的是 CSV 文件的哪一行。
#另一个 for 循环会遍历 CSV Reader 对象返回所有行，除了第一行，所有行都会添加到 csvRows。
#在 for 循环遍历每一行时，代码检查 reader.line_num 是否设为 1。
#如果是这样，它执行 continue，转向下一行，不将它添加到 csvRows 中。
#对于之后的每一行，条件永远是 False，该行将添加到 csvRows 中

#第 3 步：写入 CSV 文件，没有第一行

    csvFileObj = open(os.path.join('headerRemoved',csvFilename),'w',newline='')
    csvWriter = csv.writer(csvFileObj)
    for row in csvRows:
        csvWriter.writerow(row)
    csvFileObj.close()

#CSV Writer 对象利用 csvFilename（这也是我们在 CSV Reader 中使用的文件名）  
#将列表写入 headerRemoved 中的一个 CSV 文件。这将覆盖原来的文件。
#创建 Writer 对象后，我们循环遍历存储在 csvRows 中的子列表，将每个子列表写入该文件。
#这段代码执行后，外层 for 循环将循环到 os.listdir('.')中的下一个文件名。循环结束时，程序就结束了。



Removing header from example.csv...
Removing header from output.csv...


## 14.3 JSON 和 API

#### json 模块  
Python 的 json 模块处理了 JSON 数据字符串和 Python 值之间转换的所有细节，   
得到了 json.loads()和 json.dumps()函数。   
JSON 不能存储每一种 Python 值，它只能包含以下数据类型的值：字符串、整型、浮点型、布尔型、列表、字典和 NoneType。  
JSON 不能表示 Python 特有的对象，如 File 对象、 CSV Reader 或 Writer 对象、 Regex对象或 Selenium WebElement 对象。

### 14.4.1 用 loads()函数读取 JSON


In [22]:
stringOfJsonData = '{"name": "Zophie", "isCat": true, "miceCaught": 0,"felineIQ": null}'
import json
jsonDataAsPythonValue = json.loads(stringOfJsonData)
jsonDataAsPythonValue

{'name': 'Zophie', 'isCat': True, 'miceCaught': 0, 'felineIQ': None}

### 14.4.2 用 dumps 函数写出 JSON


In [23]:
 pythonValue = {'isCat': True, 'miceCaught': 0, 'name': 'Zophie','felineIQ': None}
import json
stringOfJsonData = json.dumps(pythonValue)
stringOfJsonData

'{"isCat": true, "miceCaught": 0, "name": "Zophie", "felineIQ": null}'

In [25]:
import json, requests, sys
# Compute location from command line arguments.
if len(sys.argv) < 2:
    print('Usage: quickWeather.py location')
    sys.exit()
location = ' '.join(sys.argv[1:])
url ='http://api.openweathermap.org/data/2.5/forecast/daily?q=%s&cnt=3' % (location)
response = requests.get(url)
response.raise_for_status()

weatherData = json.loads(response.text)
# Print weather descriptions.
w = weatherData['list']
print('Current weather in %s:' % (location))
print(w[0]['weather'][0]['main'], '-', w[0]['weather'][0]['description'])
print()
print('Tomorrow:')
print(w[1]['weather'][0]['main'], '-', w[1]['weather'][0]['description'])
print()
print('Day after tomorrow:')
print(w[2]['weather'][0]['main'], '-', w[2]['weather'][0]['description'])

HTTPError: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/data/2.5/forecast/daily?q=-f%20C:%5CUsers%5Cxingjinzhao%5CAppData%5CRoaming%5Cjupyter%5Cruntime%5Ckernel-24e429b3-5f57-45e2-b3c9-a0a06d6a609b.json&cnt=3